#  Import some necessary package

In [2]:
from __future__ import print_function

import numpy as np
import pandas as pd
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense,MaxPooling1D,Dropout,Conv1D,Embedding, Activation,GlobalMaxPooling1D
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras import utils
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Load Data

In [3]:
df=pd.read_csv('mail.csv')

In [4]:
df.sample(5)

,label,text
2895,spam,pdate_Now - Double mins and 1000 txts on Orang...
401,ham,Dear how is chechi. Did you talk to her
1044,spam,We know someone who you know that fancies you....
1993,ham,Eh den sat u book e kb liao huh...
214,ham,Sounds great! Are you home now?


In [5]:
x = df.iloc[:,1].values
y = df.iloc[:,0].values
x

array(['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'Ok lar... Joking wif u oni...',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       ..., 'Pity, * was in mood for that. So...any other suggestions?',
       "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
       'Rofl. Its true to its name'], dtype=object)

# Two label spam and nonspam or ham

In [6]:
print(np.unique(y, return_counts=True))

(array(['ham', 'spam'], dtype=object), array([4825,  747], dtype=int64))


# encode the text with word sequences 

In [7]:
 # encode the text with word sequences - Preprocessing step 1
tk = Tokenizer(num_words= 200, filters = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True, split=" ")
tk.fit_on_texts(x)
x = tk.texts_to_sequences(x)
x = sequence.pad_sequences(x, maxlen=200)

print(x)

[[  0   0   0 ...  67  58 144]
 [  0   0   0 ...   0  46   6]
 [  0   0   0 ...   2  73 179]
 ...
 [  0   0   0 ...  20  23 105]
 [  0   0   0 ...   2  12  47]
 [  0   0   0 ...  61   2  61]]


In [8]:
labels_encoder_y = LabelEncoder()
y=labels_encoder_y.fit_transform(y)
print(y)
print(np.unique(y, return_counts=True))

[0 0 1 ... 0 0 0]
(array([0, 1]), array([4825,  747], dtype=int64))


# Perform one hot encoding 

In [9]:
y=utils.to_categorical(y, num_classes=2)
y

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [10]:
x_train,x_test,y_train,y_test=train_test_split(x, y, test_size=0.2, random_state=42)

In [11]:
x_train=sequence.pad_sequences(x_train, maxlen=202)
x_test=sequence.pad_sequences(x_test, maxlen=202)

In [12]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (4457, 202)
x_test shape: (1115, 202)


In [13]:
max_features = 10000
maxlen = 201
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250

print('Build model...')
model = Sequential()

model=Sequential([
    Embedding(max_features,embedding_dims, input_length=maxlen),
    Dropout(0.2),
    Conv1D(filters, kernel_size,padding='valid',activation='relu',strides=1),
    GlobalMaxPooling1D(),
    
    Dense(hidden_dims,activation='relu'),
    Dropout(0.2),
    Dense(hidden_dims,activation='relu'),
    Dropout(0.2),
    Dense(2, activation='sigmoid')
])

Build model...


# model_summary

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 201, 50)           500000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 201, 50)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 199, 250)          37750     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_2 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 250)              

# Model_Compile

In [17]:



model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history=model.fit(x_train, y_train,
          batch_size=32,
          epochs=2,
          verbose=2,
          validation_data=(x_test, y_test))

ValueError: Error when checking input: expected embedding_1_input to have 2 dimensions, but got array with shape (4457, 1, 202)

In [ ]:
# LSTM with dropout for sequence classification 
import numpy
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.preprocessing import sequence,text
from keras.layers.embeddings import Embedding
import pandas as pd


# fix random seed for reproducibility
numpy.random.seed(7)

#fetching sms spam dataset
url = 'https://raw.githubusercontent.com/justmarkham/pydata-dc-2016-tutorial/master/sms.tsv'
sms = pd.read_table(url, header=None, names=['label', 'message'])

#binarizing
sms['label_num'] = sms.label.map({'ham':0, 'spam':1})
sms.head()

X = sms.message
y = sms.label_num
print(X.shape)
print(y.shape)

###################################
tk = text.Tokenizer(nb_words=200, lower=True)
tk.fit_on_texts(X)

x = tk.texts_to_sequences(X)

print len(tk.word_counts)

###################################
max_len = 80
print "max_len ", max_len
print('Pad sequences (samples x time)')

x = sequence.pad_sequences(x, maxlen=max_len)



max_features = 200
model = Sequential()
print('Build model...')

model = Sequential()
model.add(Embedding(max_features, 128, input_length=max_len, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop')

model.fit(x, y=y, batch_size=500, nb_epoch=1, verbose=1, validation_split=0.2, show_accuracy=True, shuffle=True)
